# Reforma (uniformización) de los resultados electorales

Ya vimos en el cuaderno re transformación inicial, que solo obteníamos las columans de los partidos que se presentaban en cada elección. Como creemos que conviene que el dataset de las cinco elecciones tengan las mismas columnas, eso es lo que conseguimos en este cuaderno.

Aprovecharemos de paso para incluir una columna adicional.

In [1]:
import pandas as pd

In [2]:
import boto3

BUCKET_NAME = 'electomedia' 

# sustituir por credenciales de acceso. 
s3 = boto3.resource('s3', aws_access_key_id = 'xxxxxxxxx', 
                          aws_secret_access_key= 'xxxxxxxxxxx')

In [3]:
import botocore.exceptions

KEY = 'INE/resultados_comp_N19.txt' # replace with your object key

try:
  # we are trying to download training dataset from s3 with name `my-training-data.csv` 
  # to colab dir with name `training.csv`
  s3.Bucket(BUCKET_NAME).download_file(KEY, 'resultados_comp_N19.txt')
  
except botocore.exceptions.ClientError as e:
  if e.response['Error']['Code'] == "404":
    print("The object does not exist.")
  else:
    raise
    

Cargamos el dataset que ya obtuvimos en el cuaderno anterior. Utilizaremos en este ejemplo los de las elecciones de noviembre de 2019.

In [4]:
res_partidos = pd.read_csv('resultados_comp_N19.txt')

In [5]:
res_partidos

,Sección,Censo_Esc,Votos_Total,Nulos,Blanco,V_Cand,BNG,Bildu,CC,CUP,...,MP,Otros,PACMA,PNV,PP,PRC,PSOE,TE,UP,VOX
0,022019111010400101001,1002,717,7,3,707,0,0,0,0,...,0,2,3,0,193,0,310,0,30,122
1,022019111010400201001,1013,711,12,1,698,0,0,0,0,...,0,2,2,0,111,0,349,0,42,147
2,022019111010400301001,667,484,7,5,472,0,0,0,0,...,0,0,3,0,176,0,128,0,34,116
3,022019111010400301002,1306,909,3,5,901,0,0,0,0,...,0,3,6,0,251,0,220,0,58,312
4,022019111010400301003,1551,975,12,9,954,0,0,0,0,...,0,3,5,0,292,0,202,0,52,327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36297,022019111195200108011,1638,1021,3,11,1007,0,0,0,0,...,0,348,0,0,303,0,140,0,28,158
36298,022019111195200108012,1676,1057,9,2,1046,0,0,0,0,...,0,97,0,0,463,0,205,0,35,210
36299,022019111195200108013,1132,638,5,4,629,0,0,0,0,...,0,108,0,0,208,0,113,0,25,144
36300,022019111195200108014,899,527,4,0,523,0,0,0,0,...,0,85,0,0,200,0,87,0,12,126


Listamos las columnas del dataset, que incluye, pero no solo, los partidos principales de las elecciones.

In [6]:
list(res_partidos)

['Sección',
 'Censo_Esc',
 'Votos_Total',
 'Nulos',
 'Blanco',
 'V_Cand',
 'BNG',
 'Bildu',
 'CC',
 'CUP',
 'Cs',
 'ERC',
 'GBai',
 'JxC',
 'MP',
 'Otros',
 'PACMA',
 'PNV',
 'PP',
 'PRC',
 'PSOE',
 'TE',
 'UP',
 'VOX']

Cargamos la lista nuestra de partidos, que será la común para las cinco elecciones. 

In [7]:
partidos = pd.read_csv('Partidos.txt', names=['Número', 'Sigla', 'Partido'])

In [8]:
partidos

,Número,Sigla,Partido
0,1,PP,Partido Popular
1,2,PSOE,Partido Socialista Obrero Español
2,3,Cs,Ciudadanos
3,4,UP,Unidas Podemos
4,5,IU,Izquierda Unida
5,6,VOX,Vox
6,7,UPyD,Unión Progreso y Democracia
7,8,MP,Más País
8,9,CiU,Convergencia i Unió
9,10,ERC,Esquerra Republicana de Catalunya


Almacenamos las siglas de los partidos en una lista.

In [9]:
lista_partidos = partidos['Sigla']

In [10]:
lista_partidos

0            PP
1          PSOE
2            Cs
3            UP
4            IU
5           VOX
6          UPyD
7            MP
8           CiU
9           ERC
10          JxC
11          CUP
12          DiL
13          PNV
14        Bildu
15       Amaiur
16           CC
17           FA
18           TE
19          BNG
20          PRC
21         GBai
22    Compromis
23        PACMA
24        Otros
Name: Sigla, dtype: object

In [11]:
lista_partidos.tolist()

['PP',
 'PSOE',
 'Cs',
 'UP',
 'IU',
 'VOX',
 'UPyD',
 'MP',
 'CiU',
 'ERC',
 'JxC',
 'CUP',
 'DiL',
 'PNV',
 'Bildu',
 'Amaiur',
 'CC',
 'FA',
 'TE',
 'BNG',
 'PRC',
 'GBai',
 'Compromis',
 'PACMA',
 'Otros']

In [12]:
lista_partidos

0            PP
1          PSOE
2            Cs
3            UP
4            IU
5           VOX
6          UPyD
7            MP
8           CiU
9           ERC
10          JxC
11          CUP
12          DiL
13          PNV
14        Bildu
15       Amaiur
16           CC
17           FA
18           TE
19          BNG
20          PRC
21         GBai
22    Compromis
23        PACMA
24        Otros
Name: Sigla, dtype: object

Ahora simplemente hacemos una resta de sets a partir de las siglas de partidos de nuestra lista, y las que aparecen en el dataset de noviembre de 2019. Como cabía esperar, formaciones como IU o UPyD ya no se presentaron entonces.

In [13]:
part = list(set(lista_partidos.tolist()) - set(list(res_partidos)))

In [14]:
part

['Amaiur', 'UPyD', 'CiU', 'FA', 'IU', 'DiL', 'Compromis']

Pero esas columnas son precisamente las que hay que añadir en este caso, aunque sean simplemente columnas llenas de ceros.

In [15]:
for item in part:
    res_partidos[item] = 0

In [16]:
res_partidos

,Sección,Censo_Esc,Votos_Total,Nulos,Blanco,V_Cand,BNG,Bildu,CC,CUP,...,TE,UP,VOX,Amaiur,UPyD,CiU,FA,IU,DiL,Compromis
0,022019111010400101001,1002,717,7,3,707,0,0,0,0,...,0,30,122,0,0,0,0,0,0,0
1,022019111010400201001,1013,711,12,1,698,0,0,0,0,...,0,42,147,0,0,0,0,0,0,0
2,022019111010400301001,667,484,7,5,472,0,0,0,0,...,0,34,116,0,0,0,0,0,0,0
3,022019111010400301002,1306,909,3,5,901,0,0,0,0,...,0,58,312,0,0,0,0,0,0,0
4,022019111010400301003,1551,975,12,9,954,0,0,0,0,...,0,52,327,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36297,022019111195200108011,1638,1021,3,11,1007,0,0,0,0,...,0,28,158,0,0,0,0,0,0,0
36298,022019111195200108012,1676,1057,9,2,1046,0,0,0,0,...,0,35,210,0,0,0,0,0,0,0
36299,022019111195200108013,1132,638,5,4,629,0,0,0,0,...,0,25,144,0,0,0,0,0,0,0
36300,022019111195200108014,899,527,4,0,523,0,0,0,0,...,0,12,126,0,0,0,0,0,0,0


In [17]:
list_partidos = lista_partidos.tolist()

Definimos una nueva columnas que, sinceramente, se nos olvidó definir en el cuaderno de la transformación inicial, se trata de los votos válidos.

In [18]:
res_partidos['Votos_Válidos'] = res_partidos['Votos_Total'] - res_partidos['Nulos']

In [19]:
res_partidos

,Sección,Censo_Esc,Votos_Total,Nulos,Blanco,V_Cand,BNG,Bildu,CC,CUP,...,UP,VOX,Amaiur,UPyD,CiU,FA,IU,DiL,Compromis,Votos_Válidos
0,022019111010400101001,1002,717,7,3,707,0,0,0,0,...,30,122,0,0,0,0,0,0,0,710
1,022019111010400201001,1013,711,12,1,698,0,0,0,0,...,42,147,0,0,0,0,0,0,0,699
2,022019111010400301001,667,484,7,5,472,0,0,0,0,...,34,116,0,0,0,0,0,0,0,477
3,022019111010400301002,1306,909,3,5,901,0,0,0,0,...,58,312,0,0,0,0,0,0,0,906
4,022019111010400301003,1551,975,12,9,954,0,0,0,0,...,52,327,0,0,0,0,0,0,0,963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36297,022019111195200108011,1638,1021,3,11,1007,0,0,0,0,...,28,158,0,0,0,0,0,0,0,1018
36298,022019111195200108012,1676,1057,9,2,1046,0,0,0,0,...,35,210,0,0,0,0,0,0,0,1048
36299,022019111195200108013,1132,638,5,4,629,0,0,0,0,...,25,144,0,0,0,0,0,0,0,633
36300,022019111195200108014,899,527,4,0,523,0,0,0,0,...,12,126,0,0,0,0,0,0,0,523


Ahora se trata de que las columnas tengan también el mismo orden, comenzamos intecalando la de votos válidos dentro de las que no son de siglas de partidos.

In [20]:
list_datos_sec = list(res_partidos)[0:6]

In [21]:
list_datos_sec

['Sección', 'Censo_Esc', 'Votos_Total', 'Nulos', 'Blanco', 'V_Cand']

In [22]:
list_datos_sec.insert(4, 'Votos_Válidos' )

In [23]:
list_datos_sec

['Sección',
 'Censo_Esc',
 'Votos_Total',
 'Nulos',
 'Votos_Válidos',
 'Blanco',
 'V_Cand']

Ahora añadimos las siglas de los partidos a la lista que definirá el orden de las columnas, que llamaremos 'orden_cols'.

In [24]:
list_datos_sec.extend(list_partidos)

In [25]:
list_datos_sec

['Sección',
 'Censo_Esc',
 'Votos_Total',
 'Nulos',
 'Votos_Válidos',
 'Blanco',
 'V_Cand',
 'PP',
 'PSOE',
 'Cs',
 'UP',
 'IU',
 'VOX',
 'UPyD',
 'MP',
 'CiU',
 'ERC',
 'JxC',
 'CUP',
 'DiL',
 'PNV',
 'Bildu',
 'Amaiur',
 'CC',
 'FA',
 'TE',
 'BNG',
 'PRC',
 'GBai',
 'Compromis',
 'PACMA',
 'Otros']

In [26]:
orden_cols = list_datos_sec

In [27]:
orden_cols

['Sección',
 'Censo_Esc',
 'Votos_Total',
 'Nulos',
 'Votos_Válidos',
 'Blanco',
 'V_Cand',
 'PP',
 'PSOE',
 'Cs',
 'UP',
 'IU',
 'VOX',
 'UPyD',
 'MP',
 'CiU',
 'ERC',
 'JxC',
 'CUP',
 'DiL',
 'PNV',
 'Bildu',
 'Amaiur',
 'CC',
 'FA',
 'TE',
 'BNG',
 'PRC',
 'GBai',
 'Compromis',
 'PACMA',
 'Otros']

Finalmente, reformamos el dataset con el orden de columnas que queremos.

In [28]:
res_partidos = res_partidos[orden_cols]

In [29]:
res_partidos

,Sección,Censo_Esc,Votos_Total,Nulos,Votos_Válidos,Blanco,V_Cand,PP,PSOE,Cs,...,Amaiur,CC,FA,TE,BNG,PRC,GBai,Compromis,PACMA,Otros
0,022019111010400101001,1002,717,7,710,3,707,193,310,47,...,0,0,0,0,0,0,0,0,3,2
1,022019111010400201001,1013,711,12,699,1,698,111,349,45,...,0,0,0,0,0,0,0,0,2,2
2,022019111010400301001,667,484,7,477,5,472,176,128,15,...,0,0,0,0,0,0,0,0,3,0
3,022019111010400301002,1306,909,3,906,5,901,251,220,51,...,0,0,0,0,0,0,0,0,6,3
4,022019111010400301003,1551,975,12,963,9,954,292,202,73,...,0,0,0,0,0,0,0,0,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36297,022019111195200108011,1638,1021,3,1018,11,1007,303,140,30,...,0,0,0,0,0,0,0,0,0,348
36298,022019111195200108012,1676,1057,9,1048,2,1046,463,205,36,...,0,0,0,0,0,0,0,0,0,97
36299,022019111195200108013,1132,638,5,633,4,629,208,113,31,...,0,0,0,0,0,0,0,0,0,108
36300,022019111195200108014,899,527,4,523,0,523,200,87,13,...,0,0,0,0,0,0,0,0,0,85


Guardamos el dataset de resultados de elecciones, en este caso 'resultados_comp_N19_ref.txt', y hacemos de forma análoga con los de las otras cuatro elecciones.

In [30]:
res_partidos.to_csv('resultados_comp_N19_ref.txt', sep = ',', index = False)